In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyarrow
from dataclasses import dataclass
import yaml
from typing import Union

In [2]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [3]:
os.chdir("..")
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [4]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Класс `DropBatchProcessor`

In [5]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

**Тест `DropBatchProcessor`**

In [2]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.processor import DropBatchHandler
from data_generator.general_time import pd_timestamp_to_unix

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_dist_cfg()
# configs = drop_cfg_build.build_purch_cfg()

base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
# base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
# base_agg1.build_all(drop_type="purchaser")
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

batch_hand = DropBatchHandler(base=base_agg1, create_txn=cr_drop_txn1)

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=3565, district_id=44, birth_date='1962-07-29', sex='male', region='Ставропольский', area='Ставрополь', timezone='UTC+3', lat=45.0445439, lon=41.9690168, population=398266, home_ip='2.60.13.50', geometry=<MULTIPOLYGON (((41.81 45.006, 41.811 45.006, 41.813 45.007, 41.815 45.007, ...>)

In [4]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data, batch_hand):
    cr_drop_txn.reset_cache(only_counters=False)
    behav_hand.reset_cache(all=True)
    amt_hand.reset_cache(all=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()
    batch_hand.reset_cache(all=True)

In [4]:
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))].sample(1)
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))]
# acc_hand1.reset_cache()
acc_hand1.used_accounts

Series([], Name: account_id, dtype: object)

**`should_decline`**  
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1

In [5]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1
batch_hand.should_decline()

True

### **Тест метода `distributor`**

**баланс > trf_max** - выводить кол-во попыток через print внутри atts_after_decline()  
4 кейса
- scen=`"split_transfer"`, in_chunks = `True`
- scen=`"atm+transfer`", in_chunks = `True`
- scen=`"atm"`, in_chunks = `False`
- scen=`"transfer"`, in_chunks = `False`

In [10]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [11]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.trf_max + 1000
    behav_hand1.scen = "transfer"
    behav_hand1.in_chunks = False

    batch_hand.distributor()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

attempts 3


In [4]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df

**С генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []

while True:
    declined = batch_hand.declined
    receive_txn1 = cr_drop_txn1.trf_or_atm(dist=True, receive=True, to_drop=False, declined=declined)
    all_txns1.append(receive_txn1)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

In [9]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df #.query("type != 'inbound'").shape

16


(11, 19)

### **Тест метода `purchaser`**

**баланс > amt_max** - выводить кол-во попыток через print внутри atts_after_decline()
- scen=`"split_money"`, in_chunks = `True`
- scen=`"transfer"`, in_chunks = `False`

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns2 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [8]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.amt_max + 1000
    behav_hand1.scen = "one_purchase"
    behav_hand1.in_chunks = False

    batch_hand.purchaser()
    txns_fm_batch2 = batch_hand.txns_fm_batch
    all_txns2.extend(txns_fm_batch2)
    batch_hand.reset_cache(all=False)

attempts 0


In [10]:
all_txns_df2 = pd.DataFrame(all_txns2)
print(all_txns_df2.shape[0])
all_txns_df2

**`purchaser` с генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns3 = []

while True:
    declined = batch_hand.declined
    receive_txn3 = cr_drop_txn1.trf_or_atm(receive=True, to_drop=False, declined=declined)
    all_txns3.append(receive_txn3)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch3 = batch_hand.txns_fm_batch
    all_txns3.extend(txns_fm_batch3)
    batch_hand.reset_cache(all=False)

In [30]:
all_txns_df3 = pd.DataFrame(all_txns3)
print(all_txns_df3.shape[0])
all_txns_df3

# from_7th = all_txns_df3.iloc[7:all_txns_df3.index.max()+1].copy()[["type", "amount", "status"]] #.query("type != 'inbound'").shape
# from_7th["balance"] = -from_7th["amount"]
# from_7th.loc[from_7th.index.min(), "balance"] = from_7th.loc[from_7th.index.min(), "amount"]
# from_7th["balance"] = from_7th["balance"].cumsum()
# from_7th

14


,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3565,2025-01-18 02:17:00,1737166620,40200.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13377.0,False,False,approved,not applicable
1,3565,2025-01-18 03:27:00,1737170820,24000.0,outbound,transfer,not applicable,True,NaN,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,18376.0,False,False,approved,not applicable
2,3565,2025-01-18 04:18:00,1737173880,16200.0,outbound,transfer,not applicable,True,NaN,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,19169.0,False,False,approved,not applicable
3,3565,2025-01-18 06:28:00,1737181680,44000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13377.0,False,False,approved,not applicable
4,3565,2025-01-19 00:33:00,1737246780,19400.0,withdrawal,ATM,not applicable,False,NaN,Ставрополь,45.044544,41.969017,not applicable,NaN,13377.0,False,False,approved,not applicable
5,3565,2025-01-19 02:23:00,1737253380,24000.0,outbound,transfer,not applicable,True,NaN,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,22998.0,False,False,approved,not applicable
6,3565,2025-01-19 04:22:00,1737260520,600.0,outbound,transfer,not applicable,True,NaN,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,16475.0,False,False,approved,not applicable
7,3565,2025-01-19 06:36:00,1737268560,49700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13377.0,False,False,approved,not applicable
8,3565,2025-01-19 08:49:00,1737276540,24000.0,purchase,crypto_exchange,balance_top_up,True,6904.0,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,NaN,False,False,approved,not applicable
9,3565,2025-01-19 11:14:00,1737285240,17000.0,outbound,transfer,not applicable,True,NaN,Ставрополь,45.044544,41.969017,2.60.13.50,6097.0,20669.0,False,False,approved,not applicable


In [29]:
amt_hand1.balance

np.float64(5700.0)

## Класс `DropLifecycleManager`
- Полный жизненный цикл дропа

In [ ]:
# ----------ВСТАВИТЬ ГОТОВЫЙ КЛАСС-------------------------

## **Тест `DropLifecycleManager`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
# from data_generator.fraud.drops.processor import DropBatchHandler
# from data_generator.general_time import pd_timestamp_to_unix
from data_generator.fraud.drops.simulator import DropLifecycleManager

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
# configs = drop_cfg_build.build_dist_cfg()
configs = drop_cfg_build.build_purch_cfg()

# base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

life_manager = DropLifecycleManager(base=base_agg1, create_txn=cr_drop_txn1)
batch_hand = life_manager.batch_hand

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=2174, district_id=74, birth_date='1952-08-08', sex='female', region='Санкт-Петербург', area='Санкт-Петербург', timezone='UTC+3', lat=59.939125, lon=30.3158225, population=4848742, home_ip='2.60.8.10', geometry=<MULTIPOLYGON (((30.043 59.764, 30.045 59.766, 30.046 59.766, 30.052 59.766,...>)

In [3]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data):
    cr_drop_txn.reset_cache()
    behav_hand.reset_cache(all=False)
    amt_hand.reset_cache(life_end=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()

**`run_drop_lifecycle`**  

In [3]:
all_txns = []
life_manager.run_drop_lifecycle()
drop_txns = life_manager.drop_txns
all_txns.extend(drop_txns)
life_manager.reset_all_caches() # сброс всего кэша после завершения активности дропа

In [4]:
all_txns_df = pd.DataFrame(all_txns)
all_txns_df

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2174,2025-01-09 20:03:00,1736452980,44400.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12057.0,False,False,approved,not applicable
1,2174,2025-01-09 20:46:00,1736455560,32000.0,purchase,ecom,misc_net,True,6840.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3714.0,NaN,False,False,approved,not applicable
2,2174,2025-01-09 23:32:00,1736465520,6000.0,purchase,ecom,misc_net,True,6871.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3713.0,NaN,False,False,approved,not applicable
3,2174,2025-01-10 02:03:00,1736474580,3000.0,purchase,ecom,travel_net,True,6777.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3713.0,NaN,False,False,approved,not applicable
4,2174,2025-01-10 04:24:00,1736483040,3000.0,purchase,ecom,misc_net,True,6927.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3713.0,NaN,False,False,approved,not applicable
5,2174,2025-01-10 07:11:00,1736493060,400.0,purchase,ecom,shopping_net,True,6942.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3713.0,NaN,False,False,approved,not applicable
6,2174,2025-01-10 22:40:00,1736548800,5300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12057.0,False,False,approved,not applicable
7,2174,2025-01-10 23:27:00,1736551620,5300.0,purchase,ecom,shopping_net,True,6804.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3713.0,NaN,False,False,approved,not applicable
8,2174,2025-01-11 01:44:00,1736559840,19200.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12057.0,False,False,approved,not applicable
9,2174,2025-01-12 01:16:00,1736644560,19200.0,purchase,ecom,shopping_net,True,6816.0,Санкт-Петербург,59.939125,30.315822,2.60.8.10,3714.0,NaN,False,False,approved,not applicable


In [5]:
behav_hand1.scen, behav_hand1.in_chunks, behav_hand1.attempts

(None, None, 0)

In [6]:
cr_drop_txn1.last_txn, cr_drop_txn1.in_txns, cr_drop_txn1.out_txns

(None, 0, 0)

In [7]:
amt_hand1.batch_txns, amt_hand1.chunk_size, amt_hand1.balance, amt_hand1.last_amt, amt_hand1.first_decl, amt_hand1.declined_txns

(0, 0, 0, 0, 0, 0)

In [8]:
time_hand1.start_unix, time_hand1.last_unix, time_hand1.in_txns, time_hand1.out_txns,

(0, 0, 0, 0)

In [9]:
part_data1.last_txn